In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.losses import mean_squared_error, binary_crossentropy
from tensorflow.keras.optimizers import Adam

In [3]:
# read in data
df = pd.read_csv("liverData/eldd.csv")

In [4]:
# print length and show sample data
print(len(df))
df.head()

654


,Age,Sex,DaysAtRisk,Deceased,LTx,Cirrhosis,ALF,Ethyltoxic,HBV,HCV,...,CYSC_S,GGT_S,IL6_S,INR_C,NA_S,P_S,PALB_S,PROT_S,PTH_S,VDT_OH_S
0,68,male,200,0,0,1.0,0,1,1,0,...,1.79,1.97,22.87,1.11,135.4,1.49,0.19,69.6,2.39,12.7
1,64,male,3,1,0,1.0,0,1,0,0,...,4.87,2.43,336.50,1.77,133.1,0.96,0.05,62.5,19.39,5.5
2,67,female,208,0,0,1.0,0,1,0,0,...,2.23,1.84,16.74,1.09,137.4,1.14,0.17,80.5,7.39,18.8
3,32,female,17,1,0,0.0,1,0,0,0,...,4.73,24.35,709.80,2.29,130.5,1.71,0.15,48.9,2.21,4.5
4,64,female,189,0,0,1.0,0,1,0,0,...,1.51,2.45,7.90,1.10,142.6,1.07,0.11,67.6,4.17,34.1


In [45]:
#meld functions
#https://www.mdcalc.com/calc/10437/model-end-stage-liver-disease-meld#evidence

import math

#meld3
#1.33*(Female) + 4.56*ln(Serum bilirubin) + 0.82*(137 - Sodium) – 0.24*(137 - Sodium)*ln(Serum bilirubin) + 9.09*ln(INR) + 11.14*ln(Serum creatinine) + 1.85*(3.5 – Serum albumin) – 1.83*(3.5 – Serum albumin)*ln(Serum creatinine) + 6,
#rounded to the nearest integer
#Serum bilirubin, INR, and serum creatinine values below 1.0 are set to 1.0.
#Sodium is limited to a range of 125-137 mEq/L, and if outside of these bounds, is set to the nearest limit.
#Serum albumin is limited to a range of 1.5-3.5 g/dL, and if outside of these bounds, is set to the nearest limit.
#Maximum serum creatinine is 3.0 mg/dL, and if above this bound, is set to 3.0 mg/dL.

def calc_meld3(CRE_S, BILI_S, INR_C, NA_S, ALB_S, sex):
    if BILI_S < 1:
        BILI_S = 1
    if INR_C < 1:
        INR_C = 1
    if CRE_S < 1:
        CRE_S = 1
    if CRE_S >= 3:
        CRE_S = 3
    if NA_S > 137:
        NA_S = 137
    if NA_S < 125:
        NA_S = 125
    if ALB_S > 3.5:
        ALB_S = 3.5
    if ALB_S < 1.5:
        ALB_S = 1.5
    if sex == 'female':
        sex = 1
    else:
        sex = 0
    
    meld3_score = 1.33*(sex) + 4.56*math.log(BILI_S) + 0.82*(137 - NA_S) - 0.24*(137 - NA_S)*math.log(BILI_S) + 9.09*math.log(INR_C) + 11.14*math.log(CRE_S) + 1.85*(3.5 - ALB_S) - 1.83*(3.5 - ALB_S)*math.log(CRE_S) + 6
    return int(round(meld3_score))

#meld_na
#MELD(i) = 0.957*ln(Creatinine) + 0.378*ln(Bilirubin) + 1.120*ln(INR) + 0.643
#Then, round to the tenth decimal place and multiply by 10. 
#If MELD(i) > 11, perform additional MELD calculation as follows:
#MELD = MELD(i) + 1.32*(137 – Na) –  [0.033*MELD(i)*(137 – Na)]

def calc_meldNA(CRE_S, BILI_S, INR_C,NA_S):
    meldNA_score = 0.957 * math.log(CRE_S) + 0.378*math.log(BILI_S) + 1.120*math.log(INR_C) + 0.643
    if meldNA_score > 11:
        meldNA_score = meldNA_score + 1.32*(137-NA_S) - (0.033*meldNA_score*(137-NA_S))
    return int(round(meldNA_score))

In [50]:
#drop NA rows to allow for meld scores. 654 down to 638
df = df.dropna(subset=['INR_C','NA_S','ALB_S'])


In [51]:
df2 = df.copy(deep=True)
df2['MELD3'] = df2.apply(lambda row: calc_meld3(row["CRE_S"],row["BILI_S"],row["INR_C"],row["NA_S"],row["ALB_S"],row["Sex"]), axis =1)
df2['MELDNA'] = df2.apply(lambda row: calc_meldNA(row["CRE_S"],row["BILI_S"],row["INR_C"],row["NA_S"]), axis =1)

In [52]:
df2.head()

,Age,Sex,DaysAtRisk,Deceased,LTx,Cirrhosis,ALF,Ethyltoxic,HBV,HCV,...,IL6_S,INR_C,NA_S,P_S,PALB_S,PROT_S,PTH_S,VDT_OH_S,MELDNA,MELD3
0,68,male,200,0,0,1.0,0,1,1,0,...,22.87,1.11,135.4,1.49,0.19,69.6,2.39,12.7,6,29
1,64,male,3,1,0,1.0,0,1,0,0,...,336.50,1.77,133.1,0.96,0.05,62.5,19.39,5.5,8,40
2,67,female,208,0,0,1.0,0,1,0,0,...,16.74,1.09,137.4,1.14,0.17,80.5,7.39,18.8,6,33
3,32,female,17,1,0,0.0,1,0,0,0,...,709.80,2.29,130.5,1.71,0.15,48.9,2.21,4.5,8,49
4,64,female,189,0,0,1.0,0,1,0,0,...,7.90,1.10,142.6,1.07,0.11,67.6,4.17,34.1,6,37
